In [10]:
import torch
from model import GPTConfig, GPT
import tiktoken
import torch.nn.functional as F

In [33]:
import torch
checkpoint_path = 'out/ckpt.pt'
checkpoint = torch.load(checkpoint_path)
checkpoint

/tmp/ipykernel_497542/3863344367.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


{'model': OrderedDict([('_orig_mod.transformer.wte.weight',
               tensor([[-0.0322, -0.0111,  0.0167,  ...,  0.0069, -0.0058, -0.0081],
                       [-0.0304,  0.0002,  0.0077,  ...,  0.0077,  0.0092, -0.0072],
                       [-0.0061,  0.0011,  0.0170,  ..., -0.0185,  0.0190,  0.0065],
                       ...,
                       [ 0.0138,  0.0087, -0.0099,  ...,  0.0040,  0.0246,  0.0086],
                       [ 0.0138,  0.0087, -0.0099,  ...,  0.0040,  0.0246,  0.0086],
                       [ 0.0138,  0.0087, -0.0099,  ...,  0.0040,  0.0246,  0.0086]],
                      device='cuda:0')),
              ('_orig_mod.transformer.wpe.weight',
               tensor([[ 7.4531e-03,  7.5798e-03,  1.9184e-03,  ...,  5.8771e-05,
                        -1.1303e-02,  1.8420e-02],
                       [-4.4606e-03, -5.1927e-03, -2.3616e-03,  ...,  6.9686e-03,
                         1.2965e-02, -1.3329e-02],
                       [-3.2076e-03, -4.165

In [27]:
model = GPT(GPTConfig(vocab_size=50304, block_size=1024))
checkpoint = torch.load(checkpoint_path)
new_pth=model.state_dict() # 需要加载参数的模型
pretrained_dict={} # 用于保存公共具有的参数
for k,v in checkpoint['model'].items():
    for kk in new_pth.keys():
        if kk in k:
            pretrained_dict[kk]=v
            break
new_pth.update(pretrained_dict)
model.load_state_dict(new_pth)

number of parameters: 123.69M


/tmp/ipykernel_497542/2221335259.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


<All keys matched successfully>

In [28]:
state_dict = checkpoint['model']
state_dict

OrderedDict([('_orig_mod.transformer.wte.weight',
              tensor([[-0.0322, -0.0111,  0.0167,  ...,  0.0069, -0.0058, -0.0081],
                      [-0.0304,  0.0002,  0.0077,  ...,  0.0077,  0.0092, -0.0072],
                      [-0.0061,  0.0011,  0.0170,  ..., -0.0185,  0.0190,  0.0065],
                      ...,
                      [ 0.0138,  0.0087, -0.0099,  ...,  0.0040,  0.0246,  0.0086],
                      [ 0.0138,  0.0087, -0.0099,  ...,  0.0040,  0.0246,  0.0086],
                      [ 0.0138,  0.0087, -0.0099,  ...,  0.0040,  0.0246,  0.0086]],
                     device='cuda:0')),
             ('_orig_mod.transformer.wpe.weight',
              tensor([[ 7.4531e-03,  7.5798e-03,  1.9184e-03,  ...,  5.8771e-05,
                       -1.1303e-02,  1.8420e-02],
                      [-4.4606e-03, -5.1927e-03, -2.3616e-03,  ...,  6.9686e-03,
                        1.2965e-02, -1.3329e-02],
                      [-3.2076e-03, -4.1653e-03, -3.1768e-03,  ...

In [29]:
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)

In [30]:
state_dict

OrderedDict([('transformer.wte.weight',
              tensor([[-0.0322, -0.0111,  0.0167,  ...,  0.0069, -0.0058, -0.0081],
                      [-0.0304,  0.0002,  0.0077,  ...,  0.0077,  0.0092, -0.0072],
                      [-0.0061,  0.0011,  0.0170,  ..., -0.0185,  0.0190,  0.0065],
                      ...,
                      [ 0.0138,  0.0087, -0.0099,  ...,  0.0040,  0.0246,  0.0086],
                      [ 0.0138,  0.0087, -0.0099,  ...,  0.0040,  0.0246,  0.0086],
                      [ 0.0138,  0.0087, -0.0099,  ...,  0.0040,  0.0246,  0.0086]],
                     device='cuda:0')),
             ('transformer.wpe.weight',
              tensor([[ 7.4531e-03,  7.5798e-03,  1.9184e-03,  ...,  5.8771e-05,
                       -1.1303e-02,  1.8420e-02],
                      [-4.4606e-03, -5.1927e-03, -2.3616e-03,  ...,  6.9686e-03,
                        1.2965e-02, -1.3329e-02],
                      [-3.2076e-03, -4.1653e-03, -3.1768e-03,  ...,  4.3417e-03,
     

In [31]:
model_args = checkpoint['model_args']
model_args['vocab_size'] = checkpoint['model_args']['vocab_size']
model = GPT(GPTConfig(**model_args))
new_pth=model.state_dict() # 需要加载参数的模型
pretrained_dict={} # 用于保存公共具有的参数
for k,v in checkpoint['model'].items():
    for kk in new_pth.keys():
        if kk in k:
            pretrained_dict[kk]=v
            break
new_pth.update(pretrained_dict)
model.load_state_dict(new_pth)
model.eval()

number of parameters: 123.59M


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50304, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=False)
          (c_proj): Linear(in_features=768, out_features=768, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=3072, out_features=768, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50304, bias=False)
)

In [17]:
from transformers import GPT2Tokenizer
enc = GPT2Tokenizer.from_pretrained('gpt2')

In [18]:
device = 'cuda'
tokens = enc.encode("Hello, I love you")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0)  # 添加批次维度
xgen = tokens.to(device)
model.to(device)


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50304, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=False)
          (c_proj): Linear(in_features=768, out_features=768, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=3072, out_features=768, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50304, bias=False)
)

In [23]:
res = model.generate(xgen, max_new_tokens=5, temperature=0.7, top_k=None)

In [24]:
res

tensor([[15496,    11,   314,  1842,   345,   366,   764,   220,   198,  1279]],
       device='cuda:0')

In [25]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.decode(res[0].tolist())

'Hello, I love you " . \n <'

In [36]:
checkpoint['iter_num']

25000